## Import the required libraries

In [ ]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import time

### Create 2 attributes 

1.   One for Text
2.   Other for label of type float



In [1]:


SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

### Download the IMDB dataset and split into 

1.   Train data
2.   Test data



In [2]:

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 79.0MB/s]


In [3]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [4]:
print(vars(train_data.examples[0]))

{'text': ['Before', 'Cujo', ',', 'there', 'was', 'Lucky', 'the', 'devil', 'dog', '.', 'In', '1978,on', 'Halloween', 'night', 'the', 'movie"Devil', 'Dog', ',', 'The', 'Hound', 'of', 'Hell', '"', 'premiered', '.', 'A', 'story', 'of', 'a', 'family', 'getting', 'a', 'new', 'puppy', '(', 'from', 'a', 'farmer', 'who', 'just', 'happen', 'to', 'be', 'in', 'the', 'neighborhood', 'selling', 'fruits', 'and', 'vegetables', ')', 'because', 'their', 'dog', 'Skipper', 'was', 'killed', '.', 'Coencidence', '?', 'Everyone', 'loves', 'the', 'new', 'dog', ',', 'but', 'there', 'is', 'something', 'strange', 'about', 'him', '.', '<', 'br', '/><br', '/>It', 'is', "n't", 'long', 'until', 'the', 'father', 'Mike', 'Barry(Richard', 'Crenna', ',', 'First', 'Blood)starts', 'to', 'notice', '.', 'His', 'wife', 'Betty(Yvette', 'Mimieux', ',', 'Where', 'The', 'Boys', 'Are', ',', 'Jackson', 'County', 'Jail', ',', 'Snowbeast)is', 'different', 'and', 'his', 'kids', 'Charlie', 'and', 'Bonnie(Ike', 'Eisenman', ',', 'Witch',

In [5]:


train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [6]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


### Initialize the Text and Label variable created above using the train data

In [7]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [8]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [9]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 203063), (',', 192343), ('.', 166129), ('and', 109794), ('a', 109526), ('of', 100808), ('to', 93971), ('is', 76428), ('in', 61581), ('I', 54306), ('it', 53609), ('that', 49177), ('"', 44610), ("'s", 43276), ('this', 42373), ('-', 36806), ('/><br', 35659), ('was', 35096), ('as', 30590), ('with', 30113)]


In [10]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [11]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


### Create iterator 

1.   Using BucketIterator




In [61]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [62]:
print(type(train_data), '-', type(valid_data), '-', type(test_data))

<class 'torchtext.legacy.data.dataset.Dataset'> - <class 'torchtext.legacy.data.dataset.Dataset'> - <class 'torchtext.legacy.datasets.imdb.IMDB'>


### Existing code for RNN model

In [66]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

### LSTM model is

*   Bi directional
*   1 layer of LSTM



In [109]:
#now define the model, here we will use LSTM which is a modified version of RNN as well as dropout as our regularizer
#LSTM will be bidirectional 

class LSTM(torch.nn.Module):
    
    def __init__(self,embed_dim,vocab_size,hid_dim,out_dim): 
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size,embed_dim)           #prepare the lookup table for word embeddings
        self.rnn = torch.nn.LSTM(embed_dim,hid_dim,bias=True,num_layers=1,bidirectional=True) #LSTM 1 layered and bidirectional
        self.fc = torch.nn.Linear(hid_dim,out_dim)          #fully connected layer for output
            
    def forward(self,feed_data):
        embed_out = self.embedding(feed_data)
        rnn_out,(rnn_hid,rnn_cell) = self.rnn(embed_out)
        hidden = rnn_hid[-1,:,:]
        return self.fc(hidden.squeeze(0))

### create a LSTM model

In [111]:

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 740
OUTPUT_DIM = 1

model = LSTM(vocab_size= INPUT_DIM ,embed_dim= EMBEDDING_DIM ,hid_dim= HIDDEN_DIM ,out_dim= OUTPUT_DIM) 


### util function to count number of parameters

In [112]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 7,485,581 trainable parameters


#### get optimizer handle

In [113]:


optimizer = optim.SGD(model.parameters(), lr=1e-3)

#### Use BCEWithLogitsLoss function

In [114]:
criterion = nn.BCEWithLogitsLoss()

In [115]:
model = model.to(device)
criterion = criterion.to(device)

#### util function to calculate model accuracy

In [116]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

### Train the model

In [117]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        #print(batch.text.shape)                
        predictions = model(batch.text).squeeze(1) #1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

### Test the model

In [118]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            #print(batch.text)
            text_lengths = torch.tensor(np.full(batch.text.shape[0],batch.text.shape[1]))
            predictions = model(batch.text).squeeze(1) #1
            #print(predictions.shape)
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [119]:


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### Train and validte the model 

In [120]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 59s
	Train Loss: 0.693 | Train Acc: 50.47%
	 Val. Loss: 0.693 |  Val. Acc: 50.45%
Epoch: 02 | Epoch Time: 1m 58s
	Train Loss: 0.693 | Train Acc: 50.66%
	 Val. Loss: 0.693 |  Val. Acc: 50.75%
Epoch: 03 | Epoch Time: 1m 56s
	Train Loss: 0.693 | Train Acc: 51.08%
	 Val. Loss: 0.693 |  Val. Acc: 51.05%
Epoch: 04 | Epoch Time: 1m 58s
	Train Loss: 0.693 | Train Acc: 51.42%
	 Val. Loss: 0.693 |  Val. Acc: 51.21%
Epoch: 05 | Epoch Time: 2m 0s
	Train Loss: 0.693 | Train Acc: 51.59%
	 Val. Loss: 0.693 |  Val. Acc: 51.28%


## The model attains a train acc of 51.59% and validation accuracy 51.28%

In [121]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.693 | Test Acc: 51.41%


## Test accuracy is 51.41%